In [1]:
import glob
import pandas as pd

In [2]:
df = pd.concat([pd.read_parquet(x) for x in glob.glob('/raid/train-1.parquet')])

In [3]:
df.head()

,hashtags,tweet_id,media,links,domains,tweet_type,language,timestamp,a_user_id,a_follower_count,...,b_account_creation,b_follows_a,reply,retweet,retweet_comment,like,id,len_hashtags,len_domains,len_links
0,None,0,0,None,None,2,11,1581262691,0,986,...,1478011810,False,0,0,0,0,0,0,0,0
1,83D6C79F5FCEC8D1CAD9E82C2C261611\tFFAD2DCF664C...,1,0,None,None,1,11,1581497241,1,1225,...,1540395738,True,0,1581497559,0,1581497622,1,5,0,0
2,None,2,0,DDFFB4C01DB85921C3580F614575AA6D,BE4539C53C53FFABCFD232DB100C792B,2,11,1580978528,2,3016,...,1432084055,True,0,0,0,1581060554,2,0,1,1
3,None,3,0,None,None,1,54,1581321849,3,2121,...,1534313747,False,0,0,0,1581328518,3,0,0,0
4,None,4,5,None,None,2,11,1580956787,4,813505,...,1490166885,False,0,0,0,1580957807,4,0,0,0


In [8]:
df['dt_dow'] = pd.to_datetime(df['timestamp'], unit='s').dt.weekday

In [9]:
%%time
# TRAIN FIRST 5 DAYS. VALIDATE LAST 2 DAYS
VALID_DOW = [1, 2]# order is [3, 4, 5, 6, 0, 1, 2]
valid = df[df['dt_dow'].isin(VALID_DOW)].reset_index(drop=True)
train = df[~df['dt_dow'].isin(VALID_DOW)].reset_index(drop=True)

CPU times: user 29.2 s, sys: 22.2 s, total: 51.3 s
Wall time: 51.3 s


In [10]:
train = train.sort_values(["b_user_id", "timestamp"]).reset_index()
valid = valid.sort_values(["b_user_id", "timestamp"]).reset_index()

In [11]:
train.shape, valid.shape

((87914868, 29), (33471563, 29))

In [12]:
train.to_parquet('/raid/org_train-1-train.parquet', row_group_size=1024)

In [13]:
valid.to_parquet('/raid/org_train-1-valid.parquet', row_group_size=1024)